In [ ]:
!pip install pygame

In [ ]:
import pygame
import random 
import math
from pygame import mixer

In [ ]:
pygame.init() #initializing the pygame

In [ ]:
screen = pygame.display.set_mode((800,600)) #screen created
background = pygame.image.load('background.png')

mixer.music.load('background.wav')
mixer.music.play(-1)

pygame.display.set_caption("Space Invaders")
icon = pygame.image.load("ufo.png")
pygame.display.set_icon(icon)


In [ ]:
#Creating Player
playerImg = pygame.image.load('player.png')
playerX = 370
playerY = 480
playerX_change = 0
playerY_change = 0

In [ ]:
#Creating Enemy
enemyImg,enemyX,enemyY,enemyY_change,enemyX_change = [],[],[],[],[]
num_of_enemies = 6

for i in range(num_of_enemies):
    enemyImg.append(pygame.image.load('enemy.png'))
    
    enemyX.append(random.randint(0,736))
    enemyY.append(random.randint(0,150))
    
    enemyX_change.append(4)
    enemyY_change.append(40)

In [ ]:
#Creating Bullet

bulletImg = pygame.image.load('bullet.png')
bulletX = playerX
bulletY = playerY

bulletX_change = 0
bulletY_change = 10

bullet_state = 'ready'

In [ ]:
#Score
score_value = 0
font = pygame.font.Font('freesansbold.ttf',32)

textX = 10
textY = 10

over_font = pygame.font.Font('freesansbold.ttf',64)

def show_score(x,y):
    score = font.render("Score : " + str(score_value),True,(255,255,255))
    screen.blit(score,(x,y))

def game_over_text():
    over_text = over_font.render("Game Over",True,(255,255,255))
    score = font.render("Score : " + str(score_value),True,(255,255,255))
    screen.blit(score,(300,300))
    screen.blit(over_text,(200,250))

In [ ]:
def player(x,y):
    screen.blit(playerImg,(x,y))
    
def enemy(x,y,i):
    screen.blit(enemyImg[i],(x,y))

In [ ]:
def fire_bullets(x,y):
    global bullet_state
    bullet_state = 'fire'
    screen.blit(bulletImg,(x,y))

In [ ]:
def isCollision(enemyX,enemyY,bulletX,bulletY):
    
    distance = math.sqrt(math.pow(enemyX - bulletX,2) + (math.pow(enemyY - bulletY,2)))
    
    if distance < 27:
        return True
    else:
        return False

In [ ]:
def isCollisionWithPlayer(enemyX,enemyY,playerX,playerY):
    
    distance1 = math.sqrt(math.pow(enemyX - playerX,2) + (math.pow(enemyY - playerY,2)))
    
    if distance1 < 50:
        return True
    else:
        return False

In [ ]:
run = True
player_dead = False

while run:
    screen.fill((0,0,0))
    screen.blit(background,(0,0))
    
    for event in pygame.event.get():
        
        if event.type == pygame.QUIT:
            run = False
    
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT and player_dead == False:
                playerX_change= -5

            if event.key == pygame.K_RIGHT and player_dead == False:
                playerX_change= 5   
                
            if event.key == pygame.K_UP and player_dead == False:
                playerY_change= -5

            if event.key == pygame.K_DOWN and player_dead == False:
                playerY_change= 5   

            if event.key == pygame.K_SPACE and player_dead == False:
                if bullet_state == 'ready':
                    bulletSound = mixer.Sound('laser.wav')
                    bulletSound.play()
                    bulletX = playerX
                    bulletY = playerY
                    fire_bullets(bulletX,bulletY)

        if event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                playerX_change = 0
            if event.key == pygame.K_UP or event.key == pygame.K_DOWN:
                playerY_change = 0

    playerX += playerX_change
    playerY += playerY_change
    
    if playerX <= 0:
        playerX = 0
        
    elif playerX>= 736:
        playerX = 736
        
    if playerY <= 0:
        playerY = 0
        
    elif playerY>= 536:
        playerY = 536
        
    
    for i in range(num_of_enemies):
        collision2 = isCollisionWithPlayer(enemyX[i],enemyY[i],playerX,playerY)
        
        if collision2:
            for j in range(num_of_enemies):
                enemyY[j] = 2000
            
            bulletX = playerX
            bulletY = playerY
            playerImg = pygame.image.load('playerdead.png')
            game_over_text()
            player_dead = True
            run = False
    
        enemyX[i] += enemyX_change[i]
        
        if enemyX[i] <= 0:
            enemyX[i] = 4
            enemyY[i] += enemyY_change[i]

        elif enemyX[i] >= 736:
            enemyX[i] = -4
            enemyY[i] += enemyY_change[i]
            
        if enemyY[i] >= 600 and player_dead == False:
            enemyY[i] = random.randint(0,150)
            enemyY[i] += enemyY_change[i]
 
        collision = isCollision(enemyX[i],enemyY[i], bulletX, bulletY)

        if collision:

            explosionSound = mixer.Sound('explosion.wav')
            explosionSound.play()

            bulletY = playerY
            
            bullet_state = 'ready'
            
            score_value +=1
            
            enemyX[i] = random.randint(0,736)
            enemyY[i] = random.randint(0,150)
            
        enemy(enemyX[i], enemyY[i], i)
        
    if bulletY <=0:
        bulletY = playerY
        bullet_state = 'ready'
            
    if bullet_state == 'fire':
            
        fire_bullets(bulletX,bulletY)
        bulletY -= bulletY_change
    
    player(playerX,playerY)
    show_score(textX,textY)
    pygame.display.update()
    